# Getting started

In this notebook, we illustrate how to use the Neural News Recommendation with Multi-Head Self-Attention ([NRMS](https://aclanthology.org/D19-1671/)). The implementation is taken from the [recommenders](https://github.com/recommenders-team/recommenders) repository. We have simply stripped the model to keep it cleaner.

We use a small dataset, which is downloaded from [recsys.eb.dk](https://recsys.eb.dk/). All the datasets are stored in the folder path ```~/ebnerd_data/*```.

## Load functionality

In [ ]:
from transformers import AutoTokenizer, AutoModel
from pathlib import Path
import tensorflow as tf
import polars as pl
import datetime
import sys 
import  ebrec
from ebrec.utils._constants import *

from ebrec.utils._behaviors import (
    create_binary_labels_column,
    sampling_strategy_wu2019,
    add_prediction_scores,
    truncate_history,
    ebnerd_from_path,
)
from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore
from ebrec.utils._articles import convert_text2encoding_with_transformers
from ebrec.utils._polars import concat_str_columns, slice_join_dataframes
from ebrec.utils._articles import create_article_id_to_value_mapping
from ebrec.utils._nlp import get_transformers_word_embeddings
from ebrec.utils._python import write_submission_file, rank_predictions_by_score
from tensorflow.keras.models import Sequential
from ebrec.models.newsrec.dataloader import NRMSDataLoader
from ebrec.models.newsrec.model_config import hparams_nrms
from ebrec.models.newsrec import NRMSModel
from tensorflow.keras.backend import clear_session
from transformers import AutoTokenizer, AutoModel
from pathlib import Path
import tensorflow as tf
import datetime as dt
import polars as pl
import numpy as np
import gc
import os

from ebrec.utils._constants import (
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_IS_BEYOND_ACCURACY_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
    DEFAULT_SUBTITLE_COL,
    DEFAULT_LABELS_COL,
    DEFAULT_TITLE_COL,
    DEFAULT_USER_COL,
)

print(sys.path)
sys.path.append(r"C:\Users\antot\Downloads\Pigasos\ebnerd-benchmark\src")


['c:\\Users\\antot\\miniconda3\\python311.zip', 'c:\\Users\\antot\\miniconda3\\DLLs', 'c:\\Users\\antot\\miniconda3\\Lib', 'c:\\Users\\antot\\miniconda3', '', 'c:\\Users\\antot\\miniconda3\\Lib\\site-packages', 'C:\\Users\\antot\\Downloads\\Pigasos\\ebnerd-benchmark\\src', 'c:\\Users\\antot\\miniconda3\\Lib\\site-packages\\win32', 'c:\\Users\\antot\\miniconda3\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\antot\\miniconda3\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\antot\\miniconda3\\Lib\\site-packages\\setuptools\\_vendor', 'C:\\Users\\antot\\Downloads\\Pigasos\\ebnerd-benchmark\\src', 'C:\\Users\\antot\\Downloads\\Pigasos\\ebnerd-benchmark\\src']


In [11]:
import ebrec
print(ebrec.__file__)


None


In [35]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.models import Sequential
print("Keras imported successfully!")


TensorFlow version: 2.15.0
Keras imported successfully!


In [36]:
# List all physical devices
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

physical_devices = tf.config.list_physical_devices()
print("Available devices:", physical_devices)

Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## Load dataset

In [37]:


# Define the data loading function
def ebnerd_from_path(path: Path, history_size: int = 30) -> pl.DataFrame:
    """
    Load ebnerd - function
    """
    df_history = (
        pl.scan_parquet(path.joinpath("history.parquet"))
        .select(DEFAULT_USER_COL, DEFAULT_HISTORY_ARTICLE_ID_COL)
        .pipe(
            truncate_history,
            column=DEFAULT_HISTORY_ARTICLE_ID_COL,
            history_size=history_size,
            padding_value=0,
            enable_warning=False,
        )
    )
    df_behaviors = (
        pl.scan_parquet(path.joinpath("behaviors.parquet"))
        .collect()
        .pipe(
            slice_join_dataframes,
            df2=df_history.collect(),
            on=DEFAULT_USER_COL,
            how="left",
        )
    )
    return df_behaviors

### Generate labels
We sample a few just to get started. For testset we just make up a dummy column with 0 and 1 - this is not the true labels.

In [ ]:
# Define main paths
TRAIN_VALID_MAIN_PATH = Path("./ebnerd_demo")
TEST_MAIN_PATH = Path("../ebnerd_testset")
DUMP_DIR = Path("ebnerd_predictions")
DUMP_DIR.mkdir(exist_ok=True, parents=True)
SEED = np.random.randint(0, 1_000)

# Define model parameters
# Replace ':' with '-' in the timestamp
MODEL_NAME = f"NRMS-{dt.datetime.now().strftime('%Y-%m-%d %H-%M-%S')}-{SEED}"
MODEL_WEIGHTS = DUMP_DIR.joinpath(f"state_dict/{MODEL_NAME}/weights")
LOG_DIR = DUMP_DIR.joinpath(f"runs/{MODEL_NAME}")
TEST_DF_DUMP = DUMP_DIR.joinpath("test_predictions", MODEL_NAME)
TEST_DF_DUMP.mkdir(parents=True, exist_ok=True)

print(f"Model Directory: {MODEL_NAME}")




History size can often be a memory bottleneck; if adjusted, the NRMS hyperparameter ```history_size``` must be updated to ensure compatibility and efficient memory usage

In [38]:

print(f"Model Directory: {MODEL_NAME}")

# Data preprocessing parameters
MAX_TITLE_LENGTH = 30
HISTORY_SIZE = 20
FRACTION = 1.0
EPOCHS = 5
FRACTION_TEST = 1.0
hparams_nrms.history_size = HISTORY_SIZE

# Batch sizes
BATCH_SIZE_TRAIN = 32
BATCH_SIZE_VAL = 32
BATCH_SIZE_TEST_WO_B = 32
BATCH_SIZE_TEST_W_B = 4
N_CHUNKS_TEST = 10
CHUNKS_DONE = 0

# Columns to select
COLUMNS = [
    DEFAULT_USER_COL,
    DEFAULT_HISTORY_ARTICLE_ID_COL,
    DEFAULT_INVIEW_ARTICLES_COL,
    DEFAULT_CLICKED_ARTICLES_COL,
    DEFAULT_IMPRESSION_ID_COL,
]

# Define train and validation paths
TRAIN_PATH = TRAIN_VALID_MAIN_PATH.joinpath("train")
VALIDATION_PATH = TRAIN_VALID_MAIN_PATH.joinpath("validation")

Model Directory: NRMS-2024-12-04 16-45-11-213


In [ ]:

# Load and preprocess the train dataset
df_train = (
    ebnerd_from_path(TRAIN_PATH, history_size=HISTORY_SIZE)
    .sample(fraction=FRACTION, seed=SEED)
    .select(COLUMNS)
    .pipe(
        sampling_strategy_wu2019,
        npratio=4,
        shuffle=True,
        with_replacement=True,
        seed=SEED,
    )
    .pipe(create_binary_labels_column)
)

# Load and preprocess the validation dataset
df_validation = (
    ebnerd_from_path(VALIDATION_PATH, history_size=HISTORY_SIZE)
    .sample(fraction=FRACTION, seed=SEED)
    .select(COLUMNS)
    .pipe(
        sampling_strategy_wu2019,
        npratio=4,
        shuffle=True,
        with_replacement=True,
        seed=SEED,
    )
    .pipe(create_binary_labels_column)
)



In [ ]:

# Initialize test data
print("Initializing test data...")

TEST_PATH = TEST_MAIN_PATH.joinpath("test")

df_test = (
    ebnerd_from_path(TEST_PATH, history_size=HISTORY_SIZE)
    .sample(fraction=FRACTION_TEST, seed=SEED)
    .with_columns(
        pl.col(DEFAULT_INVIEW_ARTICLES_COL)
        .list.first()
        .alias(DEFAULT_CLICKED_ARTICLES_COL)
    )
    .select(COLUMNS + [DEFAULT_IS_BEYOND_ACCURACY_COL])
    .with_columns(
        pl.col(DEFAULT_INVIEW_ARTICLES_COL)
        .list.eval(pl.element() * 0)
        .alias(DEFAULT_LABELS_COL)
    )
)

# Split test data based on beyond-accuracy flag
df_test_wo_beyond = df_test.filter(~pl.col(DEFAULT_IS_BEYOND_ACCURACY_COL))
df_test_w_beyond = df_test.filter(pl.col(DEFAULT_IS_BEYOND_ACCURACY_COL))

# Chunk the test data for processing
df_test_chunks = chunk_dataframe(df_test_wo_beyond, n_chunks=N_CHUNKS_TEST)
df_pred_test_wo_beyond = []

for i, df_test_chunk in enumerate(df_test_chunks[CHUNKS_DONE:], start=1 + CHUNKS_DONE):
    print(f"Processing test chunk {i}/{len(df_test_chunks)}...")
    
    # Initialize DataLoader for the current test chunk
    test_dataloader_wo_b = NRMSDataLoader(
        behaviors=df_test_chunk,
        article_dict=article_mapping,
        unknown_representation="zeros",
        history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
        eval_mode=True,
        batch_size=BATCH_SIZE_TEST_WO_B,
    )
    
    # Predict scores
    scores = model.scorer.predict(test_dataloader_wo_b)
    
    # Clear TensorFlow session to free up resources
    clear_session()
    
    # Add prediction scores to the test chunk
    df_test_chunk = add_prediction_scores(df_test_chunk, scores.tolist()).with_columns(
        pl.col("scores")
        .map_elements(lambda x: list(rank_predictions_by_score(x)))
        .alias("ranked_scores")
    )
    
    # Save the processed chunk
    df_test_chunk.select(DEFAULT_IMPRESSION_ID_COL, "ranked_scores").write_parquet(
        TEST_DF_DUMP.joinpath(f"pred_wo_ba_{i}.parquet")
    )
    
    # Append to the list of predictions
    df_pred_test_wo_beyond.append(df_test_chunk)
    
    # Cleanup
    del df_test_chunk, test_dataloader_wo_b, scores
    gc.collect()

# Concatenate all prediction chunks
df_pred_test_wo_beyond = pl.concat(df_pred_test_wo_beyond)
df_pred_test_wo_beyond.select(DEFAULT_IMPRESSION_ID_COL, "ranked_scores").write_parquet(
    TEST_DF_DUMP.joinpath("pred_wo_ba.parquet")
)

# Process test data with beyond-accuracy
print("Processing test data with beyond-accuracy...")
test_dataloader_w_b = NRMSDataLoader(
    behaviors=df_test_w_beyond,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=BATCH_SIZE_TEST_W_B,
)
scores = model.scorer.predict(test_dataloader_w_b)
df_pred_test_w_beyond = add_prediction_scores(
    df_test_w_beyond, scores.tolist()
).with_columns(
    pl.col("scores")
    .map_elements(lambda x: list(rank_predictions_by_score(x)))
    .alias("ranked_scores")
)
df_pred_test_w_beyond.select(DEFAULT_IMPRESSION_ID_COL, "ranked_scores").write_parquet(
    TEST_DF_DUMP.joinpath("pred_w_ba.parquet")
)

# Concatenate all test predictions
df_test = pl.concat([df_pred_test_wo_beyond, df_pred_test_w_beyond])
df_test.select(DEFAULT_IMPRESSION_ID_COL, "ranked_scores").write_parquet(
    TEST_DF_DUMP.joinpath("pred_concat.parquet")
)

# Generate Submission File
write_submission_file(
    impression_ids=df_test[DEFAULT_IMPRESSION_ID_COL],
    prediction_scores=df_test["ranked_scores"],
    path=DUMP_DIR.joinpath("predictions.txt"),
    filename_zip=f"{DATASPLIT}_predictions-{MODEL_NAME}.zip",
)

In this example we sample the dataset, just to keep it smaller. We'll split the training data into training and validation 

### Test set
We'll use the validation set, as the test set.

## Load articles

In [39]:

df_articles_train = pl.read_parquet(TRAIN_VALID_MAIN_PATH.joinpath("articles.parquet"))

df_articles_test = pl.read_parquet(TEST_MAIN_PATH.joinpath("articles.parquet"))

## Init model using HuggingFace's tokenizer and wordembedding
In the original implementation, they use the GloVe embeddings and tokenizer. To get going fast, we'll use a multilingual LLM from Hugging Face. 
Utilizing the tokenizer to tokenize the articles and the word-embedding to init NRMS.


In [40]:

# Define transformer parameters
TRANSFORMER_MODEL_NAME = "FacebookAI/xlm-roberta-base"
TEXT_COLUMNS_TO_USE = [DEFAULT_SUBTITLE_COL, DEFAULT_TITLE_COL]

# Load Hugging Face Transformer
transformer_model = AutoModel.from_pretrained(TRANSFORMER_MODEL_NAME)
transformer_tokenizer = AutoTokenizer.from_pretrained(TRANSFORMER_MODEL_NAME)

word2vec_embedding = get_transformers_word_embeddings(transformer_model)

# Concatenate text columns
df_articles, cat_cal = concat_str_columns(df_articles_train, columns=TEXT_COLUMNS_TO_USE)

# Convert text to encoding with transformers
df_articles, token_col_title = convert_text2encoding_with_transformers(
    df_articles, transformer_tokenizer, cat_cal, max_length=MAX_TITLE_LENGTH
)

# Create mapping from article IDs to their embeddings
article_mapping = create_article_id_to_value_mapping(
    df=df_articles, value_col=token_col_title
)



# Initiate the dataloaders
In the implementations we have disconnected the models and data. Hence, you should built a dataloader that fits your needs.

Note, with this ```NRMSDataLoader``` the ```eval_mode=False``` is meant for ```model.model.fit()``` whereas ```eval_mode=True``` is meant for ```model.scorer.predict()```. 

In [ ]:
# Initialize DataLoaders for train and validation
print("Initializing train and validation dataloaders...")

In [42]:
BATCH_SIZE = 32

train_dataloader = NRMSDataLoader(
    behaviors=df_train,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=BATCH_SIZE,
)
val_dataloader = NRMSDataLoader(
    behaviors=df_validation,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=False,
    batch_size=BATCH_SIZE,
)

## Train the model


In [43]:
# List all physical devices
physical_devices = tf.config.list_physical_devices()
print("Available devices:", physical_devices)

Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


Initiate the NRMS-model:

In [45]:
model = NRMSModel(
    hparams=hparams_nrms,
    word2vec_embedding=word2vec_embedding,
    seed=42,
)
model.model.compile(
    optimizer=model.model.optimizer,
    loss=model.model.loss,
    metrics=["AUC"],
)

MODEL_NAME = model.__class__.__name__
MODEL_WEIGHTS = DUMP_DIR.joinpath(f"state_dict/{MODEL_NAME}/weights")
LOG_DIR = DUMP_DIR.joinpath(f"runs/{MODEL_NAME}")

### Callbacks
We will add some callbacks to model training.

In [46]:
# Tensorboard:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=LOG_DIR,
    histogram_freq=1,
)

# Earlystopping:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_auc",
    mode="max",
    patience=3,
    restore_best_weights=True,
)

# ModelCheckpoint:
modelcheckpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=MODEL_WEIGHTS,
    monitor="val_auc",
    mode="max",
    save_best_only=True,
    save_weights_only=True,
    verbose=1,
)

# Learning rate scheduler:
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_auc",
    mode="max",
    factor=0.2,
    patience=2,
    min_lr=1e-6,
)

callbacks = [tensorboard_callback, early_stopping, modelcheckpoint, lr_scheduler]

In [47]:
USE_CALLBACKS = True
EPOCHS = 1

hist = model.model.fit(
    train_dataloader,
    validation_data=val_dataloader,
    epochs=EPOCHS,
    callbacks=callbacks if USE_CALLBACKS else [],
)


  2/778 [..............................] - ETA: 38:24 - loss: 3.4468 - auc: 0.5584  

KeyboardInterrupt: 

In [35]:
if USE_CALLBACKS:
    _ = model.model.load_weights(filepath=MODEL_WEIGHTS)

# Example how to compute some metrics:

In [36]:
BATCH_SIZE_TEST = 16

test_dataloader = NRMSDataLoader(
    behaviors=df_test,
    article_dict=article_mapping,
    unknown_representation="zeros",
    history_column=DEFAULT_HISTORY_ARTICLE_ID_COL,
    eval_mode=True,
    batch_size=BATCH_SIZE_TEST,
)

In [21]:
pred_test = model.scorer.predict(test_dataloader)

153/153 [==============================] - 142s 923ms/step


## Add the predictions to the dataframe

In [22]:
df_test = add_prediction_scores(df_test, pred_test.tolist())
df_test.head(2)

user_id,impression_id,impression_time,article_id_fixed,article_ids_clicked,article_ids_inview,labels,scores
u32,u32,datetime[μs],list[i32],list[i32],list[i32],list[i8],list[f64]
1823980,30777836,2023-05-26 11:08:08,"[9778769, 9777182, … 9779996]",[9782421],"[9782180, 9782421, … 9782391]","[0, 1, … 0]","[0.996318, 0.99995, … 0.999989]"
799160,41754374,2023-05-25 12:17:22,"[9779674, 9779511, … 9779860]",[9780514],"[9780856, 9780514, … 9780791]","[0, 1, … 0]","[0.999377, 0.999693, … 0.999579]"


### Compute metrics

In [23]:
metrics = MetricEvaluator(
    labels=df_test["labels"].to_list(),
    predictions=df_test["scores"].to_list(),
    metric_functions=[AucScore(), MrrScore(), NdcgScore(k=5), NdcgScore(k=10)],
)
metrics.evaluate()

AUC:   0%|                                             | 0/2446 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
AUC: 100%|███████████████████████████████| 2446/2446 [00:00<00:00, 43055.69it/s]


<MetricEvaluator class>: 
 {
    "auc": 0.5019162034296264,
    "mrr": 0.31422072947587976,
    "ndcg@5": 0.3451454383469061,
    "ndcg@10": 0.42719144948301424
}

## Make submission file

In [24]:
df_test = df_test.with_columns(
    pl.col("scores")
    .map_elements(lambda x: list(rank_predictions_by_score(x)))
    .alias("ranked_scores")
)
df_test.head(2)

user_id,impression_id,impression_time,article_id_fixed,article_ids_clicked,article_ids_inview,labels,scores,ranked_scores
u32,u32,datetime[μs],list[i32],list[i32],list[i32],list[i8],list[f64],list[i64]
1823980,30777836,2023-05-26 11:08:08,"[9778769, 9777182, … 9779996]",[9782421],"[9782180, 9782421, … 9782391]","[0, 1, … 0]","[0.996318, 0.99995, … 0.999989]","[6, 3, … 1]"
799160,41754374,2023-05-25 12:17:22,"[9779674, 9779511, … 9779860]",[9780514],"[9780856, 9780514, … 9780791]","[0, 1, … 0]","[0.999377, 0.999693, … 0.999579]","[5, 2, … 3]"


This is using the validation, simply add the testset to your flow.

In [28]:
write_submission_file(
    impression_ids=df_test[DEFAULT_IMPRESSION_ID_COL],
    prediction_scores=df_test["ranked_scores"],
    path=DUMP_DIR.joinpath("predictions.txt"),
    filename_zip=f"{DATASPLIT}_predictions-{MODEL_NAME}.zip",
)

0it [00:00, ?it/s]

2446it [00:00, 27609.70it/s]

Zipping ebnerd_predictions/predictions.txt to ebnerd_predictions/ebnerd_small_predictions-NRMSModel.zip


# DONE 🚀